#### Earthquake Data API: https://earthquake.usgs.gov/fdsnws/event/1/

##### Question1: How many earthquakes of at least 5 magnitude happened in and around Tonga in May of 2017?


In [1]:
#Import Requests Libray to interact with API/send HTTP requests
import requests
#Tonga Location ["latitude"=-21.17&longitude=-175.19]
#Count number of earthquakes within 500 km of Tonga in May 2017 with atleast 5 magnitude
query1=requests.get('https://earthquake.usgs.gov/fdsnws/event/1/count?format=text&starttime=2017-05-01&endtime=2017-05-31&minmagnitude=5&latitude=-21.17&longitude=-175.19&maxradiuskm=500')
print(query1.status_code)
print(query1.content)

200
b'7'


##### Answer1: Seven earthquakes of at least 5 magnitude happened in and around Tonga in May of 2017.

#### SpaceX Launch Data - https://github.com/r-spacex/SpaceX-API/wiki
##### 1. In which year did SpaceX first successfully reuse a capsule?
##### 2. Which flights used core Merlin 1C?
##### 3. Transform this launch data into the following relational tables and document your model


###### Question1. In which year did SpaceX first successfully reuse a capsule?
###### Approach - For any successful reuse of a capsule, reuse.capsule should be True

In [2]:
import urllib
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd
response = urlopen('https://api.spacexdata.com/v2/launches').read()
#Import json data to python dictionary
launch= json.loads(response)
#Transform back to JSON objects, launches Table is crucial as I wouldn't be duplicating or deleting data from launches
launches = pd.io.json.json_normalize(launch)

In [3]:
#From reading the details and checking the original launch, the minimum launch date is in year 2014
capsule=launches[['launch_date_utc','reuse.capsule']]
print(capsule.loc[capsule['reuse.capsule'] == True].min())

launch_date_utc    2017-06-03T21:07:00.000Z
reuse.capsule                          True
dtype: object


###### Answer 1. SpaceX first successfully reuse a capsule in 2017

##### Question 2. Which flights used core Merlin 1C?

In [4]:
#Function to flatten data, columns containing dictionaries as data will be converted to new columns
def unpack(df, column, fillna=None):
    ret = None
    if fillna is None:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems()))], axis=1)
        del ret[column]
    else:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems())).fillna(fillna)], axis=1)
        del ret[column]
    return ret

##### Approach - Some flights have more than 1 payloads, which means one core can be used in multiple flights. 
##### To solve this problem, I'm going to flatten the JSON data and append it to the original table,
##### so that I know which flights used which cores

In [5]:
response = urlopen('https://api.spacexdata.com/v2/launches').read()
#Import json data to python dictionary
launch= json.loads(response)
#Transform back to JSON objects
launches = pd.io.json.json_normalize(launch)
#launches has 55 rows 29 columns
#Unpack the row first stage cores
launchdata=unpack(launches, 'rocket.first_stage.cores')

#Get the rows containing more than 1 payloads 
launchdata1=launchdata[launchdata[1].notnull()]
launchdata2=launchdata[launchdata[2].notnull()]

#Rename the launches and drop extra so that two dataframes can be appended
launchdata1=launchdata1.drop([0,2], axis=1).rename(columns={1: 0})
launchdata2=launchdata2.drop([0,1], axis=1).rename(columns={2: 0})
#Drop the columns 1 and 2 from launch data
launchdata=launchdata.drop([1,2], axis=1)
#Append the launch data
launchdata3=launchdata.append([launchdata1, launchdata2], ignore_index=True)

#Unpack the column containing core information
launchdata4=unpack(launchdata3, 0)


#Dataframe containing flights with core_Serial 1C
rockets1C=launchdata4.loc[launchdata4['core_serial'] == 'Merlin1C']
print(rockets1C.loc[:,('flight_number','core_serial')])

   flight_number core_serial
2              3    Merlin1C


##### Answer 2. Flight number 3

##### 3. Transform this launch data into the following relational tables and document your model

In [6]:
#PAYLOAD TABLE
#Unpack the row first stage cores
launchdata6=unpack(launches, 'rocket.second_stage.payloads')

#Get the rows containing more than 1 payloads 
launchdata7=launchdata6[launchdata6[1].notnull()]

#Rename the launches and drop extra so that two dataframes can be appended
launchdata8=launchdata7.drop([0], axis=1).rename(columns={1: 0})

#Drop the columns 1 and 2 from launch data
launchdata9=launchdata6.drop([1], axis=1)
#Append the launch data
launchdata10=launchdata9.append([launchdata8], ignore_index=True)

#Unpack the column containing payload information
launchdata11=unpack(launchdata10, 0)
intermediary=launchdata11[['payload_id','rocket.rocket_id','flight_number','customers']]
intermediary=intermediary.rename(columns={'rocket.rocket_id':'rocket_id'})
intermediary['customers']=intermediary['customers'].map(lambda x: str(x)[2:-2])
payloads=intermediary[['payload_id','rocket_id','flight_number']]
print(payloads.head())

    payload_id rocket_id  flight_number
0  FalconSAT-2   falcon1              1
1      DemoSAT   falcon1              2
2  Trailblazer   falcon1              3
3       RatSat   falcon1              4
4     RazakSAT   falcon1              5


In [7]:
#Split the values of a column and expand so the new DataFrame has one split
#    value per row. Filters rows where the column is missing.
def tidy_split(df, column, sep='|', keep=False):
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    new_df.index = range(len(new_df.index))
    return new_df
intermediary1=tidy_split(intermediary, 'customers', sep=',')

In [8]:
intermediary1['customersn']=intermediary1['customers'].str.replace('\W','')
intermediary2=intermediary1.drop(['customers','rocket_id','flight_number'], axis=1)
print(intermediary2.head())

    payload_id customersn
0  FalconSAT-2      DARPA
1      DemoSAT      DARPA
2  Trailblazer       NASA
3       RatSat     SpaceX
4     RazakSAT       ATSB


In [9]:
#Rockets Table
query3=pd.read_json("https://api.spacexdata.com/v2/rockets")
rockets=query3[['id','name','type']]
rockets

,id,name,type
0,falcon1,Falcon 1,rocket
1,falcon9,Falcon 9,rocket
2,falconheavy,Falcon Heavy,rocket
3,starship,Starship,rocket


In [10]:
#Site Table
site=launches[['launch_site.site_id','launch_site.site_name_long']]
site=site.rename(columns={'launch_site.site_id':'site_id','launch_site.site_name_long':'site_name'})
#Remove [] and '' from customers
site=site.drop_duplicates()
print(site.head())

            site_id                                          site_name
0   kwajalein_atoll                      Kwajalein Atoll Omelek Island
5      ccafs_slc_40  Cape Canaveral Air Force Station Space Launch ...
10      vafb_slc_4e  Vandenberg Air Force Base Space Launch Complex 4E
35       ksc_lc_39a   Kennedy Space Center Historic Launch Complex 39A


In [11]:
customers=intermediary2[['customersn']].drop_duplicates('customersn')
print(customers.head())

  customersn
0      DARPA
2       NASA
3     SpaceX
4       ATSB
6   NASACOTS


In [12]:
#Read data from Rockets table as the rocketid don't have a unique type in the complete launch data
query3=pd.read_json("https://api.spacexdata.com/v2/rockets")
rockets=query3[['id','name','type']]
print(rockets)

            id          name    type
0      falcon1      Falcon 1  rocket
1      falcon9      Falcon 9  rocket
2  falconheavy  Falcon Heavy  rocket
3     starship      Starship  rocket


In [13]:
#Site Table
site=launches[['launch_site.site_id','launch_site.site_name_long']]
site=site.rename(columns={'launch_site.site_id':'site_id','launch_site.site_name_long':'site_name'})
#Remove [] and '' from customers
site=site.drop_duplicates()
print(site.head())

            site_id                                          site_name
0   kwajalein_atoll                      Kwajalein Atoll Omelek Island
5      ccafs_slc_40  Cape Canaveral Air Force Station Space Launch ...
10      vafb_slc_4e  Vandenberg Air Force Base Space Launch Complex 4E
35       ksc_lc_39a   Kennedy Space Center Historic Launch Complex 39A


In [14]:
#Flights Table
flights=launchdata[['flight_number','launch_site.site_id','rocket.rocket_id']]
#print(flights.head())
flights1=flights.rename(columns={'launch_site.site_id':'site_id','rocket.rocket_id':'rocket_id'})
print(flights1.head())

   flight_number          site_id rocket_id
0              1  kwajalein_atoll   falcon1
1              2  kwajalein_atoll   falcon1
2              3  kwajalein_atoll   falcon1
3              4  kwajalein_atoll   falcon1
4              5  kwajalein_atoll   falcon1


In [15]:
#PAYLOAD TABLE
#Unpack the row first stage cores
launchdata6=unpack(launches, 'rocket.second_stage.payloads')

#Get the rows containing more than 1 payloads 
launchdata7=launchdata6[launchdata6[1].notnull()]

#Rename the launches and drop extra so that two dataframes can be appended
launchdata8=launchdata7.drop([0], axis=1).rename(columns={1: 0})

#Drop the columns 1 and 2 from launch data
launchdata9=launchdata6.drop([1], axis=1)
#Append the launch data
launchdata10=launchdata9.append([launchdata8], ignore_index=True)

#Unpack the column containing payload information
launchdata11=unpack(launchdata10, 0)
intermediary=launchdata11[['payload_id','rocket.rocket_id','flight_number','customers']]
intermediary=intermediary.rename(columns={'rocket.rocket_id':'rocket_id'})
intermediary['customers']=intermediary['customers'].map(lambda x: str(x)[2:-2])
payloads=intermediary[['payload_id','rocket_id','flight_number']]
print(payloads.head())

    payload_id rocket_id  flight_number
0  FalconSAT-2   falcon1              1
1      DemoSAT   falcon1              2
2  Trailblazer   falcon1              3
3       RatSat   falcon1              4
4     RazakSAT   falcon1              5


In [16]:
intermediary=payloads[['payload_id','flight_number']]
intermediary
intermediary1['customersn']=intermediary1['customers'].str.replace('\W','')
intermediary2=intermediary1.drop(['customers','rocket_id','flight_number'], axis=1)
print(intermediary2.head())

    payload_id customersn
0  FalconSAT-2      DARPA
1      DemoSAT      DARPA
2  Trailblazer       NASA
3       RatSat     SpaceX
4     RazakSAT       ATSB


In [17]:
customers=intermediary2[['customersn']].drop_duplicates('customersn')
print(customers.head())

  customersn
0      DARPA
2       NASA
3     SpaceX
4       ATSB
6   NASACOTS


In [ ]:
#Model would have six tables
#Rockets Table {PK:ID, Name, Type}
#Flights Table {PK: Flights, FK:Site_ID, FK:Rocket_ID}
#Sites Table {PK: Site_ID, SiteName}
#Payloads Table {PK:ID, FK:Rocket_ID, FK:Flight_Number}
#Customers Table {PK: CustomerID}
#Intermediary Table between Customers and Payloads
#The Customer and Payload relationship is many to many. As payload ID and Customer ID form a unique key and 
#we will use this as a composite key, I have called this table as intermediary table

##### Skip the below

In [ ]:
#Write rockets data to rockets table in sqlite
import sqlite3
conn=sqlite3.connect('C:/Users/priya/Downloads/sqlitestudio-3.1.1/SQLiteStudio/assignment')
cur = conn.cursor()                                  
wildcards = ','.join(['?'] * len(rockets.columns))              
data = [tuple(x) for x in rockets.values]
cur.execute("drop table if exists %s" % 'rockets')
col_str = '"' + '","'.join(rockets.columns) + '"'
cur.execute("create table %s (%s)" % ('rockets', col_str)) 
cur.executemany("insert into %s values(%s)" % ('rockets', wildcards), data) 
conn.commit()
conn.close()

In [ ]:
#Write data to flights table
import sqlite3
conn=sqlite3.connect('C:/Users/priya/Downloads/sqlitestudio-3.1.1/SQLiteStudio/assignment')
cur = conn.cursor()                                 
wildcards = ','.join(['?'] * len(flights.columns))              
data = [tuple(x) for x in flights.values]
cur.execute("drop table if exists %s" % 'flights')
col_str = '"' + '","'.join(flights.columns) + '"'
cur.execute("create table %s (%s)" % ('flights', col_str)) 
cur.executemany("insert into %s values(%s)" % ('flights', wildcards), data) 
conn.commit()
conn.close()